In [9]:
# SNES-ify an existing WAV into a stereo SNES-style WAV
# Input:  /storage/emulated/0/Download/voila_music/my_loop.wav
# Output: /storage/emulated/0/Download/voila_music/my_loop_snes.wav

import wave, numpy as np
from pathlib import Path

# ---------- SETTINGS ----------
FOLDER = Path("D:\Python Virtual Environment\Digital Twin Game\Soundtrack")
INPUT_NAME = "wildwest_loop.wav"   # <-- change this to your file
IN_PATH = FOLDER / INPUT_NAME
OUT_PATH = FOLDER / (IN_PATH.stem + "_snes.wav")

TARGET_SR   = 32000   # ~SNES playback vibe (try 32000 or 32040)
LP_CUTOFF   = 8000.0  # soften highs (7–9 kHz)
CRUSH_BITS  = 11      # 10–12: lower = grittier
CH_DEPTH_MS = 5.5     # chorus depth (ms)
CH_RATE_HZ  = 0.33    # chorus LFO rate
CH_MIX      = 0.22
ECHO_MS     = 180     # delay time (ms)
ECHO_FB     = 0.42    # feedback 0..~0.6
ECHO_WET    = 0.30    # wet mix 0..~0.4

# ---------- WAV IO ----------
def read_wav_mono(path):
    with wave.open(str(path), "rb") as f:
        ch = f.getnchannels()
        sr = f.getframerate()
        sw = f.getsampwidth()
        n  = f.getnframes()
        raw = f.readframes(n)

    # support 16-bit PCM (common). For others, simple fallbacks.
    if sw == 2:
        x = np.frombuffer(raw, dtype=np.int16).astype(np.float32) / 32768.0
    elif sw == 1:  # 8-bit unsigned
        x = (np.frombuffer(raw, dtype=np.uint8).astype(np.float32) - 128) / 128.0
    else:
        # Fallback: interpret as 16-bit
        x = np.frombuffer(raw, dtype=np.int16).astype(np.float32) / 32768.0

    if ch == 2:
        x = x.reshape(-1, 2).mean(axis=1)  # downmix to mono
    return x.astype(np.float32), sr

def write_stereo_wav(path, L, R, sr):
    # tiny crossfade at ends helps seamless loops
    fade = int(0.01 * sr)
    if len(L) > 2*fade:
        w = np.linspace(0,1,fade).astype(np.float32)
        L[:fade] *= w; R[:fade] *= w
        w = np.linspace(1,0,fade).astype(np.float32)
        L[-fade:] *= w; R[-fade:] *= w

    L16 = (np.clip(L, -1, 1) * 32767).astype(np.int16)
    R16 = (np.clip(R, -1, 1) * 32767).astype(np.int16)
    inter = np.empty(L16.size + R16.size, dtype=np.int16)
    inter[0::2] = L16
    inter[1::2] = R16
    with wave.open(str(path), "wb") as f:
        f.setnchannels(2)
        f.setsampwidth(2)
        f.setframerate(sr)
        f.writeframes(inter.tobytes())

# ---------- DSP HELPERS ----------
def resample_linear(x, old_sr, new_sr):
    if new_sr == old_sr or len(x) == 0:
        return x.astype(np.float32)
    n_new = int(round(len(x) * (new_sr / old_sr)))
    xi = np.linspace(0, len(x)-1, n_new)
    x0 = np.floor(xi).astype(int)
    x1 = np.minimum(x0+1, len(x)-1)
    frac = xi - x0
    y = (1-frac)*x[x0] + frac*x[x1]
    return y.astype(np.float32)

def lowpass_simple(x, sr, cutoff_hz=8000.0):
    if len(x) == 0: return x
    rc = 1.0 / (2*np.pi*cutoff_hz)
    alpha = 1.0 / (1.0 + rc * (1.0/sr))
    y = np.empty_like(x)
    acc = 0.0
    for i, s in enumerate(x):
        acc += alpha * (s - acc)
        y[i] = acc
    return y.astype(np.float32)

def saturate_soft(x, drive=1.25):
    y = np.tanh(x * drive)
    m = np.max(np.abs(y)) or 1.0
    return (y / (m*1.02)).astype(np.float32)

def bitcrush(x, bits=11):
    levels = float(2**(bits-1))
    return (np.round(np.clip(x, -1, 1) * levels) / levels).astype(np.float32)

def chorus(x, sr, depth_ms=5.5, rate_hz=0.33, mix=0.22):
    n = len(x)
    if n == 0: return x
    depth = int(sr * depth_ms/1000.0)
    t = np.arange(n) / sr
    mod = (np.sin(2*np.pi*rate_hz*t) * 0.5 + 0.5)  # 0..1
    delay = (mod * depth).astype(int)
    y = x.copy()
    for i in range(n):
        j = i - delay[i]
        if j >= 0:
            y[i] = (1-mix)*y[i] + mix*x[j]
    return y.astype(np.float32)

def stereo_echo(xL, xR, sr, delay_ms=180, fb=0.42, wet=0.30, cross=True):
    D = int(sr * delay_ms/1000.0)
    n = len(xL)
    yL = xL.copy().astype(np.float32)
    yR = xR.copy().astype(np.float32)
    for i in range(D, n):
        fL = yR[i-D] if cross else yL[i-D]
        fR = yL[i-D] if cross else yR[i-D]
        yL[i] += wet * (xL[i-D] + fb * fL)
        yR[i] += wet * (xR[i-D] + fb * fR)
    m = max(np.max(np.abs(yL)), np.max(np.abs(yR)), 1.0)
    yL /= (m*1.02); yR /= (m*1.02)
    return yL.astype(np.float32), yR.astype(np.float32)

def snesify_mono_to_stereo(x_mono, sr_in=44100,
                           target_sr=32000,
                           lp_cut=8000.0,
                           crush_bits=11,
                           chorus_depth_ms=5.5,
                           chorus_rate_hz=0.33,
                           chorus_mix=0.22,
                           echo_delay_ms=180,
                           echo_fb=0.42,
                           echo_wet=0.30):
    # 1) downsample + lowpass
    x = resample_linear(x_mono, sr_in, target_sr)
    x = lowpass_simple(x, target_sr, lp_cut)
    # 2) gentle saturation + bit-crush
    x = saturate_soft(x, drive=1.25)
    x = bitcrush(x, bits=crush_bits)
    # 3) make L/R with slightly different chorus (detune) for width
    L = chorus(x, target_sr, depth_ms=chorus_depth_ms,     rate_hz=chorus_rate_hz,     mix=chorus_mix)
    R = chorus(x, target_sr, depth_ms=chorus_depth_ms*0.9, rate_hz=chorus_rate_hz*1.08, mix=chorus_mix)
    L *= 1.03; R *= 0.97  # tiny static pan gain offset
    # 4) stereo echo w/ cross-feedback for SPC-like space
    L, R = stereo_echo(L, R, target_sr, delay_ms=echo_delay_ms, fb=echo_fb, wet=echo_wet, cross=True)
    return L, R, target_sr

# ---------- RUN ----------
if not IN_PATH.exists():
    raise FileNotFoundError(f"Input WAV not found: {IN_PATH}")

mono, sr_in = read_wav_mono(IN_PATH)
L, R, snes_sr = snesify_mono_to_stereo(
    mono, sr_in=sr_in,
    target_sr=TARGET_SR,
    lp_cut=LP_CUTOFF,
    crush_bits=CRUSH_BITS,
    chorus_depth_ms=CH_DEPTH_MS,
    chorus_rate_hz=CH_RATE_HZ,
    chorus_mix=CH_MIX,
    echo_delay_ms=ECHO_MS,
    echo_fb=ECHO_FB,
    echo_wet=ECHO_WET
)
write_stereo_wav(OUT_PATH, L, R, snes_sr)
print("SNES-style stereo saved to:", OUT_PATH)


<>:9: SyntaxWarning: invalid escape sequence '\P'
<>:9: SyntaxWarning: invalid escape sequence '\P'
C:\Users\beach\AppData\Local\Temp\ipykernel_3464\3126041020.py:9: SyntaxWarning: invalid escape sequence '\P'
  FOLDER = Path("D:\Python Virtual Environment\Digital Twin Game\Soundtrack")


SNES-style stereo saved to: D:\Python Virtual Environment\Digital Twin Game\Soundtrack\wildwest_loop_snes.wav


In [ ]:
from PIL import Image

# Open your image
img = Image.open("vegas_1958.png")

# Resize by max width/height (e.g., 1200px max)
img.thumbnail((1200, 1200))

# Save a smaller version
img.save("vegas_1958_small.png", optimize=True, quality=85)

print("Saved downsized image as vegas_1958_small.png")